In [83]:
import pandas as pd
import numpy as np
from zipfile import ZipFile
import re
pd.options.display.max_columns = None
pd.options.display.max_rows = 20

In [84]:
!kaggle datasets download -d yagunnersya/fifa-21-messy-raw-dataset-for-cleaning-exploring

Traceback (most recent call last):
  File "/usr/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
ModuleNotFoundError: No module named 'kaggle'


In [85]:
zip_name = "fifa-21-messy-raw-dataset-for-cleaning-exploring.zip"

with ZipFile(file=zip_name, mode='r') as file:
    file.extractall()

In [86]:
data_csv = "fifa21 raw data v2.csv"
data = pd.read_csv(data_csv, engine='python')

# Data Preprocessing

In [87]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18979 entries, 0 to 18978
Data columns (total 77 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                18979 non-null  int64 
 1   Name              18979 non-null  object
 2   LongName          18979 non-null  object
 3   photoUrl          18979 non-null  object
 4   playerUrl         18979 non-null  object
 5   Nationality       18979 non-null  object
 6   Age               18979 non-null  int64 
 7   ↓OVA              18979 non-null  int64 
 8   POT               18979 non-null  int64 
 9   Club              18979 non-null  object
 10  Contract          18979 non-null  object
 11  Positions         18979 non-null  object
 12  Height            18979 non-null  object
 13  Weight            18979 non-null  object
 14  Preferred Foot    18979 non-null  object
 15  BOV               18979 non-null  int64 
 16  Best Position     18979 non-null  object
 17  Joined      

In [88]:
data.describe(include="object")

,Name,LongName,photoUrl,playerUrl,Nationality,Club,Contract,Positions,Height,Weight,Preferred Foot,Best Position,Joined,Loan Date End,Value,Wage,Release Clause,W/F,SM,A/W,D/W,IR,Hits
count,18979,18979,18979,18979,18979,18979,18979,18979,18979,18979,18979,18979,18979,1013,18979,18979,18979,18979,18979,18979,18979,18979,16384
unique,17920,18852,18979,18979,164,682,131,640,62,79,2,15,1869,24,255,134,1216,5,5,3,3,5,439
top,J. Rodríguez,Danny Rose,https://cdn.sofifa.com/players/158/023/21_60.png,http://sofifa.com/player/158023/lionel-messi/2...,England,No Club,2019 ~ 2021,CB,180cm,70kg,Right,CB,"Jul 1, 2019","Jun 30, 2021",€1.2M,€2K,€0,3 ★,2★,Medium,Medium,1 ★,1
freq,13,3,1,1,1705,237,1706,2441,1474,1495,14445,3686,1344,770,582,2899,1261,11695,9142,12701,13956,17629,2337


## Convert datetime column "Joined"

In [89]:
data.Joined.value_counts()

Jul 1, 2019     1344
Jul 1, 2018      865
Jan 1, 2019      682
Jul 1, 2017      520
Jul 1, 2020      435
                ... 
May 17, 2018       1
Jun 1, 2005        1
Mar 29, 2015       1
Feb 19, 2018       1
Mar 6, 2018        1
Name: Joined, Length: 1869, dtype: int64

In [90]:
month_dict = {
        'Jan': 1,
        'Feb': 2,
        'Mar': 3,
        'Apr': 4,
        'May': 5,
        'Jun': 6,
        'Jul': 7,
        'Aug': 8,
        'Sep': 9,
        'Oct': 10,
        'Nov': 11,
        'Dec': 12,
    }

def convertDay(date: str):
    month, day, year = re.findall(pattern=r"\w+", string=date)
    month = month_dict.get(month)
    if month/10 < 1: month = f"0{month}"
    if len(day) < 2: day = f"0{day}"
    return f"{year}-{month}-{day}"

data.Joined = data.Joined.apply(convertDay)

In [91]:
data.Joined.value_counts()

2019-07-01    1344
2018-07-01     865
2019-01-01     682
2017-07-01     520
2020-07-01     435
              ... 
2018-05-17       1
2005-06-01       1
2015-03-29       1
2018-02-19       1
2018-03-06       1
Name: Joined, Length: 1869, dtype: int64

## Converting Height

In [92]:
# Overview of Height 
data.Height.value_counts()

180cm    1474
178cm    1250
185cm    1182
183cm    1148
175cm    1091
         ... 
6'5"        1
157cm       1
5'4"        1
5'6"        1
155cm       1
Name: Height, Length: 62, dtype: int64

In [93]:
# Rows not containing "cm" 
print("There are {num} rows not containing \"cm\".".format(num=data.Height[~data.Height.str.contains(r"cm")].shape[0]))
data.Height[~data.Height.str.contains(r"cm")].value_counts()

There are 40 rows not containing "cm".


6'0"     8
5'11"    6
6'2"     4
6'3"     4
5'10"    4
6'1"     3
5'9"     3
5'7"     3
6'4"     2
6'5"     1
5'6"     1
5'4"     1
Name: Height, dtype: int64

In [94]:
# Convert height in feet to cm
def convertHeight(height: str):
    feet, inch = pd.to_numeric(re.findall(pattern='\d+', string=height))
    return str(round(feet*30.48 + inch*2.54))

data.Height[data.Height.str.contains(r"'")] = data.Height[data.Height.str.contains(r"'")].apply(convertHeight)

/tmp/ipykernel_138757/1498851119.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.Height[data.Height.str.contains(r"'")] = data.Height[data.Height.str.contains(r"'")].apply(convertHeight)


In [95]:
# Remove "cm" and convert height to numeric
data.Height = data.Height.str.replace('cm', '')
data.Height = pd.to_numeric(data.Height)
data.Height

0        170
1        187
2        188
3        181
4        175
        ... 
18974    178
18975    175
18976    179
18977    175
18978    188
Name: Height, Length: 18979, dtype: int64

## Converting Weight

In [96]:
data.Weight.value_counts()

70kg      1495
75kg      1457
80kg      1108
72kg      1022
78kg       991
          ... 
190lbs       1
130lbs       1
146lbs       1
203lbs       1
157lbs       1
Name: Weight, Length: 79, dtype: int64

In [97]:
# Rows not containing "kg" 
print("There are {num} rows not containing \"cm\".".format(num=data.Weight[~data.Weight.str.contains(r"kg")].shape[0]))
data.Weight[~data.Weight.str.contains(r"kg")].value_counts()

There are 40 rows not containing "cm".


172lbs    4
170lbs    4
183lbs    3
179lbs    3
165lbs    2
         ..
146lbs    1
203lbs    1
185lbs    1
176lbs    1
163lbs    1
Name: Weight, Length: 23, dtype: int64

In [98]:
# Convert weight in lbs to kg
def convertWeight(weight: str):
    lbs = pd.to_numeric(re.findall(pattern='\d+', string=weight))[0]
    return str(round(lbs*0.453592))

data.Weight[data.Weight.str.contains(r"lbs")] = data.Weight[data.Weight.str.contains(r"lbs")].apply(convertWeight)

/tmp/ipykernel_138757/2083810685.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.Weight[data.Weight.str.contains(r"lbs")] = data.Weight[data.Weight.str.contains(r"lbs")].apply(convertWeight)


In [99]:
# Remove "kg" and convert weight to numeric
data.Weight = data.Weight.str.replace('kg', '')
data.Weight = pd.to_numeric(data.Weight)
data.Weight

0        72
1        83
2        87
3        70
4        68
         ..
18974    66
18975    65
18976    74
18977    69
18978    75
Name: Weight, Length: 18979, dtype: int64

## Remove redundant characters in Club

In [100]:
data.Club.value_counts()

No Club                           237
\n\n\n\nFC Barcelona               33
\n\n\n\nBurnley                    33
\n\n\n\nRC Celta                   33
\n\n\n\nTorino                     33
                                 ... 
\n\n\n\nMelbourne Victory          18
\n\n\n\nBrisbane Roar              18
\n\n\n\nAdelaide United            18
\n\n\n\nPerth Glory                18
\n\n\n\nCentral Coast Mariners     18
Name: Club, Length: 682, dtype: int64

In [101]:
# Remove "\n"
data.Club = data.Club.str.replace('\n\n\n\n', '')

In [102]:
# Records that start with number, which is abnormal
data.Club[data.Club.str.contains(r"^\d+")].value_counts()

1. FSV Mainz 05          32
1. FC Köln               31
1. FC Nürnberg           30
1. FC Heidenheim 1846    30
1. FC Union Berlin       28
1. FC Kaiserslautern     28
1. FC Magdeburg          28
1. FC Saarbrücken        27
Name: Club, dtype: int64

In [103]:
# Remove "1. " from those
data.Club = data.Club.apply(
    lambda x: pd.Series(re.sub(pattern=r"^\d+\. ", string=x, repl="")))

## Reformat Contract
- Split "On Loan" to another column.
- join "Date End Loan" to Contract records having On Loan = 1.

In [104]:
data.Contract.value_counts()

2019 ~ 2021            1706
2020 ~ 2022            1445
2020 ~ 2021            1440
2019 ~ 2022            1236
2018 ~ 2021            1163
                       ... 
Jul 5, 2021 On Loan       1
2006 ~ 2024               1
2020 ~ 2026               1
2010 ~ 2025               1
Aug 1, 2021 On Loan       1
Name: Contract, Length: 131, dtype: int64

In [105]:
# Rows with different patterns
data.Contract[~data.Contract.str.contains(r"\d ~ \d")].value_counts()

Jun 30, 2021 On Loan    770
Free                    237
Dec 31, 2020 On Loan    100
May 31, 2021 On Loan     30
Dec 31, 2021 On Loan     22
                       ... 
Jan 17, 2021 On Loan      1
Jul 1, 2021 On Loan       1
Jul 5, 2021 On Loan       1
Jan 30, 2021 On Loan      1
Aug 1, 2021 On Loan       1
Name: Contract, Length: 25, dtype: int64

In [106]:
# Split "On Loan" as 1 for rows containing "On Loan" 
data['On Loan'] = data.Contract.apply(
    lambda x: 1 if str(x).find("On Loan") != -1 else 0
)

In [107]:
data['On Loan'].value_counts()

0    17966
1     1013
Name: On Loan, dtype: int64

In [108]:
data['Loan Date End'].value_counts()

Jun 30, 2021    770
Dec 31, 2020    100
May 31, 2021     30
Dec 31, 2021     22
Jan 31, 2021     18
               ... 
Jun 30, 2023      1
Jul 31, 2021      1
Dec 30, 2020      1
Jan 30, 2021      1
Aug 1, 2021       1
Name: Loan Date End, Length: 24, dtype: int64

In [109]:
# Join year of "Loan Date End" to "Contract", forming a consistent pattern of Year - Year.
data.Contract = data.apply(
    lambda row: 
        "{start} - {end}".format(
            start=re.findall(pattern=r'\w+', string=row['Contract'])[2],
            end=re.findall(pattern=r'\w+', string=row['Loan Date End'])[2])
        if f"On Loan" in str(row['Contract']) else row['Contract'],
        axis =1
)

data.Contract = data.Contract.str.replace('~', '-')

In [110]:
data.Contract.value_counts()

2019 - 2021    1706
2020 - 2022    1445
2020 - 2021    1440
2019 - 2022    1236
2018 - 2021    1163
               ... 
2012 - 2028       1
2002 - 2020       1
2019 - 2026       1
2006 - 2024       1
2009 - 2025       1
Name: Contract, Length: 110, dtype: int64

## Convert currency columns (Value, Wage, Release Cause)

In [111]:
data.Value

0        €103.5M
1           €63M
2          €120M
3          €129M
4          €132M
          ...   
18974      €100K
18975      €130K
18976      €120K
18977      €100K
18978      €100K
Name: Value, Length: 18979, dtype: object

In [112]:
def convertMoney(money: str):
    value = pd.to_numeric(re.findall(pattern=r"\d+\.?\d+|\d", string=money))[0]
    if value == np.nan: value = 0
    if "M" in money:
        value *= 1000000
    elif "K" in money:
        value *= 1000
    return str(int(value))
    

data.Value = pd.to_numeric(data.Value.apply(convertMoney))
data.Wage = pd.to_numeric(data.Wage.apply(convertMoney))
data['Release Clause'] = pd.to_numeric(data['Release Clause'].apply(convertMoney))
    

In [113]:
data.Value

0        103500000
1         63000000
2        120000000
3        129000000
4        132000000
           ...    
18974       100000
18975       130000
18976       120000
18977       100000
18978       100000
Name: Value, Length: 18979, dtype: int64

## Remove special character 
## Representing skill-based features (W/F, SM, IR)

In [114]:
data['W/F']

0        4 ★
1        4 ★
2        3 ★
3        5 ★
4        5 ★
        ... 
18974    2 ★
18975    2 ★
18976    2 ★
18977    3 ★
18978    3 ★
Name: W/F, Length: 18979, dtype: object

In [115]:
data['W/F'] = data['W/F'].apply(lambda x: re.sub(pattern="[^\d]", repl="", string=x))
data['SM'] = data['SM'].apply(lambda x: re.sub(pattern="[^\d]", repl="", string=x))
data['IR'] = data['IR'].apply(lambda x: re.sub(pattern="[^\d]", repl="", string=x))


## Reorder values in "Position"

In [116]:
positions = set()

all_pos = data.Positions.unique()
for value in all_pos:
    pos = re.findall(pattern=r"\w+", string=value)
    positions.update(pos)

print(f"There are {len(positions)} different positions")
print(positions)
    

There are 15 different positions
{'LB', 'ST', 'CM', 'CDM', 'LWB', 'RW', 'CAM', 'GK', 'CF', 'RB', 'CB', 'LW', 'LM', 'RM', 'RWB'}


Due to a large number of possible positions (15), splitting them into 15 different columns is not a good idea. I would just sort them in alphabetical order, each position is seperated with a space.

In [117]:
def reorderPositions(positions: str):
    splitted = re.findall(pattern=r"\w+", string=positions)
    splitted.sort()
    res = " ".join(splitted)
    return res

data.Positions = data.Positions.apply(reorderPositions)

In [125]:
data.head()

,ID,Name,Nationality,Age,↓OVA,POT,Club,Contract,Positions,Height,Weight,Preferred Foot,BOV,Best Position,Joined,On Loan,Value,Wage,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,W/F,SM,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits
0,158023,Lionel Messi,Argentina,33,93,93,FC Barcelona,2004 - 2021,CF RW ST,170,72,Left,93,RW,2004-07-01,0,103500000,560000,429,85,95,70,91,88,470,96,93,94,91,96,451,91,80,91,94,95,389,86,68,72,69,94,347,44,40,93,95,75,96,91,32,35,24,54,6,11,15,14,8,2231,466,4,4,Medium,Low,5,85,92,91,95,38,65,771
1,20801,C. Ronaldo dos Santos Aveiro,Portugal,35,92,92,Juventus,2018 - 2022,LW ST,187,83,Right,92,ST,2018-07-10,0,63000000,220000,437,84,95,90,82,86,414,88,81,76,77,92,431,87,91,87,95,71,444,94,95,84,78,93,353,63,29,95,82,84,95,84,28,32,24,58,7,11,15,14,11,2221,464,4,5,High,Low,5,89,93,81,89,35,77,562
2,200389,Jan Oblak,Slovenia,27,91,93,Atlético Madrid,2014 - 2023,GK,188,87,Right,91,GK,2014-07-16,0,120000000,125000,95,13,11,15,43,13,109,12,13,14,40,30,307,43,60,67,88,49,268,59,78,41,78,12,140,34,19,11,65,11,68,57,27,12,18,437,87,92,78,90,90,1413,489,3,1,Medium,Medium,3,87,92,78,90,52,90,150
3,192985,Kevin De Bruyne,Belgium,29,91,91,Manchester City,2015 - 2023,CAM CM,181,70,Right,91,CAM,2015-08-30,0,129000000,370000,407,94,82,55,94,82,441,88,85,83,93,92,398,77,76,78,91,76,408,91,63,89,74,91,408,76,66,88,94,84,91,186,68,65,53,56,15,13,5,10,13,2304,485,5,4,High,High,4,76,86,93,88,64,78,207
4,190871,Neymar da Silva Santos Jr.,Brazil,28,91,91,Paris Saint-Germain,2017 - 2022,CAM LW,175,68,Right,91,LW,2017-08-03,0,132000000,270000,408,85,87,62,87,87,448,95,88,89,81,95,453,94,89,96,91,83,357,80,62,81,50,84,356,51,36,87,90,92,93,94,35,30,29,59,9,9,15,15,11,2175,451,5,5,High,Medium,5,91,85,86,94,36,59,595


## Reordering and drop insightless features

In [119]:
cols = data.columns.tolist()
print(cols)

['ID', 'Name', 'LongName', 'photoUrl', 'playerUrl', 'Nationality', 'Age', '↓OVA', 'POT', 'Club', 'Contract', 'Positions', 'Height', 'Weight', 'Preferred Foot', 'BOV', 'Best Position', 'Joined', 'Loan Date End', 'Value', 'Wage', 'Release Clause', 'Attacking', 'Crossing', 'Finishing', 'Heading Accuracy', 'Short Passing', 'Volleys', 'Skill', 'Dribbling', 'Curve', 'FK Accuracy', 'Long Passing', 'Ball Control', 'Movement', 'Acceleration', 'Sprint Speed', 'Agility', 'Reactions', 'Balance', 'Power', 'Shot Power', 'Jumping', 'Stamina', 'Strength', 'Long Shots', 'Mentality', 'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure', 'Defending', 'Marking', 'Standing Tackle', 'Sliding Tackle', 'Goalkeeping', 'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes', 'Total Stats', 'Base Stats', 'W/F', 'SM', 'A/W', 'D/W', 'IR', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY', 'Hits', 'On Loan']


In [120]:
# Reorder the columns and drop 'Name', 'Loan Date End', 'Joined', 'photoUrl','playerUrl', 'Release Clause'
data = data[['ID', 'LongName', 'Nationality', 'Age', 'OVA', 'POT', 'Club', 
             'Contract', 'Positions', 'Height', 'Weight', 'Preferred Foot', 'BOV', 'Best Position',
             'Joined', 'On Loan',
             'Value', 'Wage', 'Attacking', 'Crossing', 'Finishing', 
             'Heading Accuracy', 'Short Passing', 'Volleys', 'Skill', 'Dribbling', 'Curve', 'FK Accuracy', 
             'Long Passing', 'Ball Control', 'Movement', 'Acceleration', 'Sprint Speed', 'Agility', 'Reactions', 
             'Balance', 'Power', 'Shot Power', 'Jumping', 'Stamina', 'Strength', 'Long Shots', 'Mentality', 
             'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure', 'Defending', 
             'Marking', 'Standing Tackle', 'Sliding Tackle', 'Goalkeeping', 'GK Diving', 'GK Handling', 
             'GK Kicking', 'GK Positioning', 'GK Reflexes', 'Total Stats', 'Base Stats', 'W/F', 'SM', 'A/W', 
             'D/W', 'IR', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY', 'Hits']]

In [133]:
data.rename(columns={'LongName': 'Name', 'OVA': 'Overall', 'POT': 'Potential', 'BOV': 'Best Overall', 'IR':'Reputation'}, inplace=True)

In [126]:
data.to_excel(excel_writer="Fifa21_Cleaned.xlsx", sheet_name="Data")

In [134]:
data.to_csv(path_or_buf="Fifa21_Cleaned.csv")